In [1]:
!pip install opensoundscape==0.7
from opensoundscape.preprocess.preprocessors import BasePreprocessor

import torch
import pandas as pd
from pathlib import Path
import numpy as np
import pandas as pd
from opensoundscape.torch.models import cnn
from opensoundscape.torch.architectures import cnn_architectures

In [22]:
df = pd.read_csv('./XenoCanto_Data/xeno_canto.csv', sep=';')
df['IN FILE'] = [f.split('/')[-1] for f in df['FILENAME']]
print(df["FILENAME"])

0        /media/jacob/Audio/Training_Xeno_Canto_2022/He...
1        /media/jacob/Audio/Training_Xeno_Canto_2022/He...
2        /media/jacob/Audio/Training_Xeno_Canto_2022/He...
3        /media/jacob/Audio/Training_Xeno_Canto_2022/He...
4        /media/jacob/Audio/Training_Xeno_Canto_2022/He...
                               ...                        
10605    /media/jacob/Audio/Training_Xeno_Canto_2022/Vi...
10606    /media/jacob/Audio/Training_Xeno_Canto_2022/Vi...
10607    /media/jacob/Audio/Training_Xeno_Canto_2022/Vi...
10608    /media/jacob/Audio/Training_Xeno_Canto_2022/Vi...
10609    /media/jacob/Audio/Training_Xeno_Canto_2022/Vi...
Name: FILENAME, Length: 10610, dtype: object


In [3]:
ef = pd.read_csv('./XenoCanto_Data/mixed_bird_manual.csv')
gtClips = np.array(ef['IN FILE'].unique())
ef.shape

(31365, 14)

In [4]:
j = 0
ff = pd.DataFrame(columns=df.columns)
for i, row in df.iterrows():
    if(row['IN FILE'] in gtClips):
        # print(row['IN FILE'])
        ff.loc[j] = row
        df.drop(i, inplace=True)
        j += 1
print(df.shape)
df = df[df['Confidence'] >= 0.8]

(10089, 8)


In [5]:
ff

,FILENAME,CLIP LENGTH,Start (s),End (s),Scientific name,Common name,Confidence,IN FILE
0,/media/jacob/Audio/Training_Xeno_Canto_2022/He...,33.515104,3.0,6.0,Hemitriccus griseipectus,White-bellied Tody-Tyrant,0.958464,Hemitriccus-griseipectus-11801.wav
1,/media/jacob/Audio/Training_Xeno_Canto_2022/He...,33.515104,6.0,9.0,Hemitriccus griseipectus,White-bellied Tody-Tyrant,0.973620,Hemitriccus-griseipectus-11801.wav
2,/media/jacob/Audio/Training_Xeno_Canto_2022/He...,33.515104,9.0,12.0,Hemitriccus griseipectus,White-bellied Tody-Tyrant,0.965499,Hemitriccus-griseipectus-11801.wav
3,/media/jacob/Audio/Training_Xeno_Canto_2022/He...,33.515104,12.0,15.0,Hemitriccus griseipectus,White-bellied Tody-Tyrant,0.946824,Hemitriccus-griseipectus-11801.wav
4,/media/jacob/Audio/Training_Xeno_Canto_2022/He...,33.515104,15.0,18.0,Hemitriccus griseipectus,White-bellied Tody-Tyrant,0.992666,Hemitriccus-griseipectus-11801.wav
...,...,...,...,...,...,...,...,...
516,/media/jacob/Audio/Training_Xeno_Canto_2022/Vi...,39.720000,36.0,39.0,Vireo olivaceus,Red-eyed Vireo,0.596736,Vireo-olivaceus-381598.wav
517,/media/jacob/Audio/Training_Xeno_Canto_2022/Vi...,11.389396,0.0,3.0,Vireo chivi,Chivi Vireo,0.938404,Vireo-olivaceus-8545.wav
518,/media/jacob/Audio/Training_Xeno_Canto_2022/Vi...,11.389396,3.0,6.0,Pycnonotus jocosus,Red-whiskered Bulbul,0.182475,Vireo-olivaceus-8545.wav
519,/media/jacob/Audio/Training_Xeno_Canto_2022/Vi...,11.389396,6.0,9.0,Vireo chivi,Chivi Vireo,0.662427,Vireo-olivaceus-8545.wav


In [6]:
len(df['FILENAME'].unique())

473

In [7]:
df['IN FILE'] = ['./XenoCanto_Data/Training_Xeno_Canto_2022/' + f for f in df['IN FILE']]
ff['IN FILE'] = ['./XenoCanto_Data/Training_Xeno_Canto_2022/' + f for f in ff['IN FILE']]

C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [8]:
ef = ff.groupby('IN FILE', group_keys=False).apply(lambda ff: ff.sample(1))
ef.columns

Index(['FILENAME', 'CLIP LENGTH', 'Start (s)', 'End (s)', 'Scientific name',
       'Common name', 'Confidence', 'IN FILE'],
      dtype='object')

In [10]:
from opensoundscape.audio import Audio
clip_duration = 3
clip_overlap = 0
final_clip = None
clip_dir = './temp_clips'
# classes = labels
min_label_overlap = 0.1

ef_split_save = pd.DataFrame(columns=['file', 'start_time', 'end_time', 'SAMPLING RATE', 'MANUAL ID'])

cnt = 0

for i, row in ef.iterrows():
    try:
        audio = Audio.from_file(row['IN FILE'])
    except Exception:
        print(row['IN FILE'] + ' not found')
        continue
    file_name = row['IN FILE'].split('.')
    # print(file_name)
    clip_df = audio.split_and_save(
        clip_dir,
        prefix=file_name[1][1:],
        clip_duration=clip_duration,
        clip_overlap=clip_overlap,
        final_clip='remainder',
        dry_run=False
    )
    clip_df['SAMPLING RATE'] = [44100] * clip_df.shape[0]
    clip_df['MANUAL ID'] = [row['Common name']] * clip_df.shape[0]
    clip_df.reset_index(inplace=True)
    # print(clip_df.shape)
    ef_split_save = ef_split_save.append(clip_df, ignore_index=True)
    # print(all_labels.shape)
    # print('*******')
print(ef_split_save)
ef_split_save.to_csv('xeno_canto_test_data.csv')

C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


                                                  file  start_time   end_time  \
0    ./temp_clips/XenoCanto_Data/Training_Xeno_Cant...         0.0   3.000000   
1    ./temp_clips/XenoCanto_Data/Training_Xeno_Cant...         3.0   6.000000   
2    ./temp_clips/XenoCanto_Data/Training_Xeno_Cant...         6.0   9.000000   
3    ./temp_clips/XenoCanto_Data/Training_Xeno_Cant...         9.0  12.000000   
4    ./temp_clips/XenoCanto_Data/Training_Xeno_Cant...        12.0  15.000000   
..                                                 ...         ...        ...   
616  ./temp_clips/XenoCanto_Data/Training_Xeno_Cant...        39.0  39.720000   
617  ./temp_clips/XenoCanto_Data/Training_Xeno_Cant...         0.0   3.000000   
618  ./temp_clips/XenoCanto_Data/Training_Xeno_Cant...         3.0   6.000000   
619  ./temp_clips/XenoCanto_Data/Training_Xeno_Cant...         6.0   9.000000   
620  ./temp_clips/XenoCanto_Data/Training_Xeno_Cant...         9.0  11.389388   

    SAMPLING RATE          

In [27]:
df = pd.read_csv('./XenoCanto_Data/xeno_canto.csv', sep=';')
df['IN FILE'] = [f.split('/')[-1] for f in df['FILENAME']]
df['IN FILE'] = ['./XenoCanto_Data/Training_Xeno_Canto_2022/' + f for f in df['IN FILE']]
df

,FILENAME,CLIP LENGTH,Start (s),End (s),Scientific name,Common name,Confidence,IN FILE
0,/media/jacob/Audio/Training_Xeno_Canto_2022/He...,29.753479,0.0,3.0,Picus awokera,Japanese Woodpecker,0.317242,./XenoCanto_Data/Training_Xeno_Canto_2022/Hemi...
1,/media/jacob/Audio/Training_Xeno_Canto_2022/He...,29.753479,3.0,6.0,Picus awokera,Japanese Woodpecker,0.505273,./XenoCanto_Data/Training_Xeno_Canto_2022/Hemi...
2,/media/jacob/Audio/Training_Xeno_Canto_2022/He...,29.753479,6.0,9.0,Picus awokera,Japanese Woodpecker,0.312248,./XenoCanto_Data/Training_Xeno_Canto_2022/Hemi...
3,/media/jacob/Audio/Training_Xeno_Canto_2022/He...,29.753479,9.0,12.0,Picus awokera,Japanese Woodpecker,0.414672,./XenoCanto_Data/Training_Xeno_Canto_2022/Hemi...
4,/media/jacob/Audio/Training_Xeno_Canto_2022/He...,29.753479,12.0,15.0,Picus awokera,Japanese Woodpecker,0.574978,./XenoCanto_Data/Training_Xeno_Canto_2022/Hemi...
...,...,...,...,...,...,...,...,...
10605,/media/jacob/Audio/Training_Xeno_Canto_2022/Vi...,112.509396,96.0,99.0,Vireo chivi,Chivi Vireo,0.116824,./XenoCanto_Data/Training_Xeno_Canto_2022/Vire...
10606,/media/jacob/Audio/Training_Xeno_Canto_2022/Vi...,112.509396,99.0,102.0,Vireo olivaceus,Red-eyed Vireo,0.568682,./XenoCanto_Data/Training_Xeno_Canto_2022/Vire...
10607,/media/jacob/Audio/Training_Xeno_Canto_2022/Vi...,112.509396,102.0,105.0,Vireo olivaceus,Red-eyed Vireo,0.301558,./XenoCanto_Data/Training_Xeno_Canto_2022/Vire...
10608,/media/jacob/Audio/Training_Xeno_Canto_2022/Vi...,112.509396,105.0,108.0,Vireo chivi,Chivi Vireo,0.269658,./XenoCanto_Data/Training_Xeno_Canto_2022/Vire...


In [28]:
def fix_folder_path(filepath):
    folder = "./XenoCanto_Data/Training_Xeno_Canto_2022/"
    filename = filepath.split("/")[-1]
    return folder + filename
df["IN FILE"] = df["file"].apply(fix_folder_path)

KeyError: 'file'

In [45]:
from opensoundscape.audio import Audio
clip_duration = 3
clip_overlap = 0
final_clip = None
clip_dir = './temp_clips'
# classes = labels
min_label_overlap = 0.1

df_split_save = pd.DataFrame(columns=['file', 'start_time', 'end_time', 'SAMPLING RATE', 'MANUAL ID', "CONFIDENCE"])

cnt = 0

for i, row in df.iterrows():
    try:
        audio = Audio.from_file(row['IN FILE'])
    except Exception:
        print(row['IN FILE'] + ' not found')
        continue
    file_name = row['IN FILE'].split('.')
    # print(file_name)
    clip_df = audio.split_and_save(
        clip_dir,
        prefix=file_name[1][1:],
        clip_duration=clip_duration,
        clip_overlap=clip_overlap,
        final_clip='remainder',
        dry_run=False
    )
    clip_df['SAMPLING RATE'] = [44100] * clip_df.shape[0]
    clip_df['MANUAL ID'] = [row['Common name']] * clip_df.shape[0]
    clip_df.reset_index(inplace=True)
    # print(clip_df.shape)
    df_split_save = df_split_save.append(clip_df, ignore_index=True)
    # print(all_labels.shape)
    # print('*******')
print(df_split_save)


                                                     file  start_time  \
0       ./temp_clips/XenoCanto_Data/Training_Xeno_Cant...         0.0   
1       ./temp_clips/XenoCanto_Data/Training_Xeno_Cant...         3.0   
2       ./temp_clips/XenoCanto_Data/Training_Xeno_Cant...         6.0   
3       ./temp_clips/XenoCanto_Data/Training_Xeno_Cant...         9.0   
4       ./temp_clips/XenoCanto_Data/Training_Xeno_Cant...        12.0   
...                                                   ...         ...   
460749  ./temp_clips/XenoCanto_Data/Training_Xeno_Cant...        99.0   
460750  ./temp_clips/XenoCanto_Data/Training_Xeno_Cant...       102.0   
460751  ./temp_clips/XenoCanto_Data/Training_Xeno_Cant...       105.0   
460752  ./temp_clips/XenoCanto_Data/Training_Xeno_Cant...       108.0   
460753  ./temp_clips/XenoCanto_Data/Training_Xeno_Cant...       111.0   

          end_time SAMPLING RATE            MANUAL ID CONFIDENCE  
0         3.000000         44100  Japanese Woodpecker   

In [55]:
df

,FILENAME,CLIP LENGTH,Start (s),End (s),Scientific name,Common name,Confidence,IN FILE,merge file
0,/media/jacob/Audio/Training_Xeno_Canto_2022/He...,29.753479,0.0,3.0,Picus awokera,Japanese Woodpecker,0.317242,./XenoCanto_Data/Training_Xeno_Canto_2022/Hemi...,/XenoCanto_Data/Training_Xeno_Canto_2022/Hemit...
1,/media/jacob/Audio/Training_Xeno_Canto_2022/He...,29.753479,3.0,6.0,Picus awokera,Japanese Woodpecker,0.505273,./XenoCanto_Data/Training_Xeno_Canto_2022/Hemi...,/XenoCanto_Data/Training_Xeno_Canto_2022/Hemit...
2,/media/jacob/Audio/Training_Xeno_Canto_2022/He...,29.753479,6.0,9.0,Picus awokera,Japanese Woodpecker,0.312248,./XenoCanto_Data/Training_Xeno_Canto_2022/Hemi...,/XenoCanto_Data/Training_Xeno_Canto_2022/Hemit...
3,/media/jacob/Audio/Training_Xeno_Canto_2022/He...,29.753479,9.0,12.0,Picus awokera,Japanese Woodpecker,0.414672,./XenoCanto_Data/Training_Xeno_Canto_2022/Hemi...,/XenoCanto_Data/Training_Xeno_Canto_2022/Hemit...
4,/media/jacob/Audio/Training_Xeno_Canto_2022/He...,29.753479,12.0,15.0,Picus awokera,Japanese Woodpecker,0.574978,./XenoCanto_Data/Training_Xeno_Canto_2022/Hemi...,/XenoCanto_Data/Training_Xeno_Canto_2022/Hemit...
...,...,...,...,...,...,...,...,...,...
10605,/media/jacob/Audio/Training_Xeno_Canto_2022/Vi...,112.509396,96.0,99.0,Vireo chivi,Chivi Vireo,0.116824,./XenoCanto_Data/Training_Xeno_Canto_2022/Vire...,/XenoCanto_Data/Training_Xeno_Canto_2022/Vireo...
10606,/media/jacob/Audio/Training_Xeno_Canto_2022/Vi...,112.509396,99.0,102.0,Vireo olivaceus,Red-eyed Vireo,0.568682,./XenoCanto_Data/Training_Xeno_Canto_2022/Vire...,/XenoCanto_Data/Training_Xeno_Canto_2022/Vireo...
10607,/media/jacob/Audio/Training_Xeno_Canto_2022/Vi...,112.509396,102.0,105.0,Vireo olivaceus,Red-eyed Vireo,0.301558,./XenoCanto_Data/Training_Xeno_Canto_2022/Vire...,/XenoCanto_Data/Training_Xeno_Canto_2022/Vireo...
10608,/media/jacob/Audio/Training_Xeno_Canto_2022/Vi...,112.509396,105.0,108.0,Vireo chivi,Chivi Vireo,0.269658,./XenoCanto_Data/Training_Xeno_Canto_2022/Vire...,/XenoCanto_Data/Training_Xeno_Canto_2022/Vireo...


In [105]:
pd.set_option('display.width', 10000)
pd.set_option('display.expand_frame_repr', False)
df["merge file"] = df["IN FILE"].apply(lambda x: x.split(".")[1])
df_split_save['CONFIDENCE'] = df_split_save['CONFIDENCE'].fillna(-1)
for i in range(df.shape[0]):
    if (i % 100 == 0):
        print(i)
    row = df.iloc[i]
    rows_to_change = (df_split_save["file"].str.contains(row["merge file"])) & (df_split_save["start_time"] == (row["Start (s)"])) & (df_split_save["end_time"] == (row["End (s)"]))
    df_split_save.loc[rows_to_change, "CONFIDENCE"]= row["Confidence"]
df_split_save

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600


,file,start_time,end_time,SAMPLING RATE,MANUAL ID,CONFIDENCE
0,./temp_clips/XenoCanto_Data/Training_Xeno_Cant...,0.0,3.000000,44100,Japanese Woodpecker,0.317242
1,./temp_clips/XenoCanto_Data/Training_Xeno_Cant...,3.0,6.000000,44100,Japanese Woodpecker,0.505273
2,./temp_clips/XenoCanto_Data/Training_Xeno_Cant...,6.0,9.000000,44100,Japanese Woodpecker,0.312248
3,./temp_clips/XenoCanto_Data/Training_Xeno_Cant...,9.0,12.000000,44100,Japanese Woodpecker,0.414672
4,./temp_clips/XenoCanto_Data/Training_Xeno_Cant...,12.0,15.000000,44100,Japanese Woodpecker,0.574978
...,...,...,...,...,...,...
460749,./temp_clips/XenoCanto_Data/Training_Xeno_Cant...,99.0,102.000000,44100,Plumbeous Vireo,0.568682
460750,./temp_clips/XenoCanto_Data/Training_Xeno_Cant...,102.0,105.000000,44100,Plumbeous Vireo,0.301558
460751,./temp_clips/XenoCanto_Data/Training_Xeno_Cant...,105.0,108.000000,44100,Plumbeous Vireo,0.269658
460752,./temp_clips/XenoCanto_Data/Training_Xeno_Cant...,108.0,111.000000,44100,Plumbeous Vireo,0.134753


In [106]:
df_split_save.drop_duplicates()

C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,file,start_time,end_time,SAMPLING RATE,MANUAL ID,CONFIDENCE
0,./temp_clips/XenoCanto_Data/Training_Xeno_Cant...,0.0,3.000000,44100,Japanese Woodpecker,0.317242
1,./temp_clips/XenoCanto_Data/Training_Xeno_Cant...,3.0,6.000000,44100,Japanese Woodpecker,0.505273
2,./temp_clips/XenoCanto_Data/Training_Xeno_Cant...,6.0,9.000000,44100,Japanese Woodpecker,0.312248
3,./temp_clips/XenoCanto_Data/Training_Xeno_Cant...,9.0,12.000000,44100,Japanese Woodpecker,0.414672
4,./temp_clips/XenoCanto_Data/Training_Xeno_Cant...,12.0,15.000000,44100,Japanese Woodpecker,0.574978
...,...,...,...,...,...,...
460749,./temp_clips/XenoCanto_Data/Training_Xeno_Cant...,99.0,102.000000,44100,Plumbeous Vireo,0.568682
460750,./temp_clips/XenoCanto_Data/Training_Xeno_Cant...,102.0,105.000000,44100,Plumbeous Vireo,0.301558
460751,./temp_clips/XenoCanto_Data/Training_Xeno_Cant...,105.0,108.000000,44100,Plumbeous Vireo,0.269658
460752,./temp_clips/XenoCanto_Data/Training_Xeno_Cant...,108.0,111.000000,44100,Plumbeous Vireo,0.134753


In [115]:
import os
from os import listdir
from os.path import isfile, join
test_path = "./temp_clips/XenoCanto_Data/Testing_Xeno_Canto_2022"
test_files = [f for f in listdir(test_path) if isfile(join(test_path, f))]
train_path = "./temp_clips/XenoCanto_Data/Training_Xeno_Canto_2022"
train_files = [f for f in listdir(train_path) if isfile(join(train_path, f))]
overlapping_files = []
for item in train_files:
    if (item in test_files):
        path = train_path + "/" + item
        os.remove(path)
        overlapping_files.append(path)
print(path)
df_split_save2 = df_split_save[~df_split_save["file"].isin(overlapping_files)]
df_split_save2.to_csv('xeno_canto_train_data.csv')       
print(len(overlapping_files))
df_split_save2

C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


./temp_clips/XenoCanto_Data/Training_Xeno_Canto_2022/Vireo-olivaceus-8545_9.0s_11.38938775510204s.wav
621


,file,start_time,end_time,SAMPLING RATE,MANUAL ID,CONFIDENCE
0,./temp_clips/XenoCanto_Data/Training_Xeno_Cant...,0.0,3.000000,44100,Japanese Woodpecker,0.317242
1,./temp_clips/XenoCanto_Data/Training_Xeno_Cant...,3.0,6.000000,44100,Japanese Woodpecker,0.505273
2,./temp_clips/XenoCanto_Data/Training_Xeno_Cant...,6.0,9.000000,44100,Japanese Woodpecker,0.312248
3,./temp_clips/XenoCanto_Data/Training_Xeno_Cant...,9.0,12.000000,44100,Japanese Woodpecker,0.414672
4,./temp_clips/XenoCanto_Data/Training_Xeno_Cant...,12.0,15.000000,44100,Japanese Woodpecker,0.574978
...,...,...,...,...,...,...
460749,./temp_clips/XenoCanto_Data/Training_Xeno_Cant...,99.0,102.000000,44100,Plumbeous Vireo,0.568682
460750,./temp_clips/XenoCanto_Data/Training_Xeno_Cant...,102.0,105.000000,44100,Plumbeous Vireo,0.301558
460751,./temp_clips/XenoCanto_Data/Training_Xeno_Cant...,105.0,108.000000,44100,Plumbeous Vireo,0.269658
460752,./temp_clips/XenoCanto_Data/Training_Xeno_Cant...,108.0,111.000000,44100,Plumbeous Vireo,0.134753


In [116]:
test_path2 = "./temp_clips/XenoCanto_Data/Testing_Xeno_Canto_2022"
test_files2 = [f for f in listdir(test_path2) if isfile(join(test_path2, f))]
train_path2 = "./temp_clips/XenoCanto_Data/Training_Xeno_Canto_2022"
train_files2 = [f for f in listdir(train_path2) if isfile(join(train_path2, f))]
overlapping_files2 = []
for item in train_files2:
    if (item in test_files2):
        overlapping_files2.append(item)
overlapping_files2

C:\Users\Siloux\.conda\envs\species-id\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[]

In [125]:
df_split_save2 = df_split_save[~df_split_save["file"].isin(overlapping_files)]
df_split_save2.to_csv('xeno_canto_train_data.csv')       
print(len(overlapping_files))

621


In [126]:
df_split_save2.loc[df_split_save2["CONFIDENCE"] == -1, "CONFIDENCE"] = 0
df_split_save2.to_csv('xeno_canto_train_data.csv') 

In [121]:
df_split_save2

0

In [42]:
for fname in overlapping_files:
    os.remove(train_path + "/" + fname)